In [127]:
# RBH via iranges 

## note: pintersect() is a parallelized intersect

library(pafr)
library(dplyr)
library(tidyr)
library(GenomicRanges)

In [128]:

## load paf 1 and 2, which are reciprocal
# cnames_paf = c("qname", "qlen	qstart	qend	strand	tname	tlen	tstart	tend	nmatch	alen	mapq")
# paf1 = read_tsv("resources/pafs/mMyoAui1.cleaned.hapheader_mMyoCai1.cleaned.hapheader.exact.paf", col_names=cnames_paf)
# paf2 = read_tsv("resources/pafs/mMyoAui1.cleaned.hapheader_mMyoCai1.cleaned.hapheader.exact.paf", col_names=cnames_paf)

paf1 <- read_paf("resources/pafs/trimmed/mMyoAui1.cleaned.hapheader_mMyoCai1.cleaned.hapheader.exact_trimmed.paf", include_tags = FALSE)
paf2 <- read_paf("resources/pafs/trimmed/mMyoCai1.cleaned.hapheader_mMyoAui1.cleaned.hapheader.exact_trimmed.paf", include_tags = FALSE)


In [129]:
# ## Species 1: intersecting RBHs

# forward_spec1 = paf1 %>% 
#   mutate(ID = paste(qname, qstart, qend, row_number())) %>%
#   select(chr = qname, 
#          start = qstart, 
#          end = qend, 
#          name = ID)

# reverse_spec1 = paf2 %>% 
#   mutate(ID = paste(tname, tstart, tend, row_number())) %>%
#   select(chr = tname, 
#          start = tstart, 
#          end = tend, 
#          name = ID)

# forward_spec1 <- makeGRangesFromDataFrame(forward_spec1, keep.extra.columns = TRUE)
# reverse_spec1 <- makeGRangesFromDataFrame(reverse_spec1, keep.extra.columns = TRUE)

In [131]:
## Species 2 intersect:

subj_spec2 = paf1 %>% 
  mutate(ID = paste(tname, tstart, tend, row_number())) %>%
  select(chr = tname, 
         start = tstart, 
         end = tend, 
         name = ID)

query_spec2 = paf2 %>% 
  mutate(ID = paste(qname, qstart, qend, row_number())) %>%
  select(chr = qname, 
         start = qstart, 
         end = qend, 
         name = ID)

spec2_tranges <- makeGRangesFromDataFrame(subj_spec2, keep.extra.columns = TRUE)
spec2_qranges <- makeGRangesFromDataFrame(query_spec2, keep.extra.columns = TRUE)
spec2_intersect <- suppressWarnings(
  GenomicRanges::intersect(spec2_tranges, spec2_qranges)
)


In [132]:
# # find the overlaps
# my_overlaps <- suppressWarnings(findOverlaps(query = spec2_qranges, subject = spec2_tranges))
 
# # create the match
# my_query <- queryHits(my_overlaps)
# my_subject <- subjectHits(my_overlaps)

# overlap_mapping <- data.frame(subject = mcols(spec2_tranges)[my_subject,],
#                               query =  mcols(spec2_qranges)[my_query,])

In [134]:
foo = function(x, y) {
     rv = x
     start(rv) = max(start(x), start(y))
     end(rv) = min(end(x), end(y))
     return(rv)
## Ref: https://bioinformatics.stackexchange.com/questions/874/intersection-of-two-genomic-ranges-to-keep-metadata
}

In [136]:
o <- findOverlaps(query = spec2_tranges, subject = spec2_intersect)
grl1 <- split(spec2_tranges[queryHits(o)], 1:length(o))
grl2 <- split(spec2_intersect[subjectHits(o)], 1:length(o))
spec2_interesct_subj <- unlist(mendoapply(foo, grl1, y=grl2))

In [138]:
o <- findOverlaps(query = spec2_qranges, subject = spec2_intersect)
grl1 <- split(spec2_qranges[queryHits(o)], 1:length(o))
grl2 <- split(spec2_intersect[subjectHits(o)], 1:length(o))
spec2_interesct_query <- unlist(mendoapply(foo, grl1, y=grl2))

In [167]:
temp_query <- data.frame(Name = spec2_interesct_query@elementMetadata$name, 
                         Chr = sapply(strsplit(spec2_interesct_query@elementMetadata$name, " "), "[", 1), 
                         Start = spec2_interesct_query@ranges@start, 
                         Width = spec2_interesct_query@ranges@width)
temp_subj <- data.frame(Name = spec2_interesct_subj@elementMetadata$name, 
                        Chr = sapply(strsplit(spec2_interesct_subj@elementMetadata$name, " "), "[", 1), 
                        Start = spec2_interesct_subj@ranges@start, 
                        Width = spec2_interesct_subj@ranges@width)
temp_query <- temp_query %>%
  dplyr::mutate(End = Start + Width-1) %>%
  dplyr::select(-Width)
temp_subj <- temp_subj %>%
  dplyr::mutate(End = Start + Width-1) %>%
  dplyr::select(-Width)                                                                 

In [168]:
head(temp_query[order(temp_query$Chr, temp_query$Start),])

,Name,Chr,Start,End
,<chr>,<chr>,<int>,<dbl>
10,mMyoAui1.0.SUPER__1 1880000 2032756 27,mMyoAui1.0.SUPER__1,1880000,2032756
20,mMyoAui1.0.SUPER__1 2096296 3224500 46,mMyoAui1.0.SUPER__1,2096296,3224500
11,mMyoAui1.0.SUPER__1 3520028 4109999 29,mMyoAui1.0.SUPER__1,3520028,4109999
19,mMyoAui1.0.SUPER__1 4290000 5550000 45,mMyoAui1.0.SUPER__1,4290000,5503973
23,mMyoAui1.0.SUPER__1 5776528 10069520 50,mMyoAui1.0.SUPER__1,5776528,10051113
18,mMyoAui1.0.SUPER__1 10071189 11199472 44,mMyoAui1.0.SUPER__1,10142466,11172295


In [169]:
head(temp_subj[order(temp_subj$Chr, temp_subj$Start),])

,Name,Chr,Start,End
,<chr>,<chr>,<int>,<dbl>
425,mMyoAui1.0.SUPER__1 1877450 2032837 404,mMyoAui1.0.SUPER__1,1880000,2032756
43,mMyoAui1.0.SUPER__1 2096096 3236718 41,mMyoAui1.0.SUPER__1,2096296,3224500
34,mMyoAui1.0.SUPER__1 3453615 4165927 31,mMyoAui1.0.SUPER__1,3520028,4109999
42,mMyoAui1.0.SUPER__1 4289594 5503973 40,mMyoAui1.0.SUPER__1,4290000,5503973
44,mMyoAui1.0.SUPER__1 5776192 10051113 42,mMyoAui1.0.SUPER__1,5776528,10051113
38,mMyoAui1.0.SUPER__1 10142466 11172295 36,mMyoAui1.0.SUPER__1,10142466,11172295
